In [2]:
# ===============================
# 1️⃣ IMPORTS
# ===============================
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import performance_metrics, cross_validation
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import warnings
warnings.filterwarnings('ignore')

# ===============================
# 2️⃣ LOAD DATA
# ===============================
df = pd.read_csv("final_df_binary.csv", parse_dates=['date'])
df = df.sort_values(['section_id', 'date']).reset_index(drop=True)

# ===============================
# 3️⃣ FEATURE ENGINEERING
# ===============================
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['dayofweek'] = df['date'].dt.dayofweek
df['quarter'] = df['date'].dt.quarter
df['is_working_day'] = ((df['is_holiday'] == 0) & (df['is_weekend'] == 0)).astype(int)

# ===============================
# 4️⃣ TRAIN/TEST SPLIT
# ===============================
train_df = df[df['year'] <= 2023].copy()
test_df = df[df['year'] == 2024].copy()

# ===============================
# 5️⃣ SECTION LIST
# ===============================
sections = df['section_id'].unique()

# ===============================
# 6️⃣ FUNCTION TO TUNE PROPHET HYPERPARAMS WITH OPTUNA
# ===============================
def tune_prophet(train_section_df, n_trials=70):
    section_id = train_section_df['section_id'].iloc[0]
    print(f"[Optuna] Tuning section: {section_id}")

    def objective(trial):
        # Prophet dataframe
        df_prophet = train_section_df.rename(columns={'date':'ds', 'total_task_time_minutes':'y'})
        changepoint_prior_scale = trial.suggest_float('changepoint_prior_scale', 0.0001, 0.5, log=True)
        seasonality_prior_scale = trial.suggest_float('seasonality_prior_scale', 0.01, 41.0, log=True)
        holidays_prior_scale = trial.suggest_float('holidays_prior_scale', 0.01, 41.0, log=True)

        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=changepoint_prior_scale,
            seasonality_prior_scale=seasonality_prior_scale,
            holidays_prior_scale=holidays_prior_scale
        )
        # Add regressors
        model.add_regressor('is_weekend')
        model.add_regressor('is_holiday')

        model.fit(df_prophet[['ds','y','is_weekend','is_holiday']])

        # Cross-validation
        df_cv = cross_validation(model, horizon='90 days', period='30 days', initial='730 days', parallel="threads")
        df_p = performance_metrics(df_cv, rolling_window=0.1)
        return df_p['mae'].values[0]

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    return study.best_params

# ===============================
# 7️⃣ SECTION-WISE MODELING & FORECASTING
# ===============================
results_dict = {}
stats_dict = {}

for sec in sections:
    print(f"\n[Main Loop] Processing section: {sec}")
    # Prepare section data
    train_sec = train_df[train_df['section_id']==sec].copy()
    test_sec = test_df[test_df['section_id']==sec].copy()

    # Tune Prophet hyperparameters
    best_params = tune_prophet(train_sec)
    print(f"{sec} best params:", best_params)

    # Fit final model
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=best_params['changepoint_prior_scale'],
        seasonality_prior_scale=best_params['seasonality_prior_scale'],
        holidays_prior_scale=best_params['holidays_prior_scale']
    )
    model.add_regressor('is_weekend')
    model.add_regressor('is_holiday')
    model.fit(train_sec[['date','total_task_time_minutes','is_weekend','is_holiday']].rename(
        columns={'date':'ds','total_task_time_minutes':'y'}
    ))

    # Predict 2024
    forecast = model.predict(test_sec[['date','is_weekend','is_holiday']].rename(columns={'date':'ds'}))
    test_sec['predicted_task_minutes'] = forecast['yhat'].values
    test_sec['error'] = test_sec['total_task_time_minutes'] - test_sec['predicted_task_minutes']

    # Store section results
    results_dict[sec] = test_sec[['date','is_weekend','is_holiday','total_task_time_minutes','predicted_task_minutes','error']].rename(
        columns={'total_task_time_minutes':'actual_task_minutes'}
    )

    # Section-wise stats
    train_stats = {
        'mean_minutes': train_sec['total_task_time_minutes'].mean(),
        'median_minutes': train_sec['total_task_time_minutes'].median(),
        'max_minutes': train_sec['total_task_time_minutes'].max(),
        'abs_error_mean': np.nan
    }
    test_stats = {
        'mean_minutes': test_sec['total_task_time_minutes'].mean(),
        'median_minutes': test_sec['total_task_time_minutes'].median(),
        'max_minutes': test_sec['total_task_time_minutes'].max(),
        'abs_error_mean': test_sec['error'].abs().mean()
    }
    stats_dict[sec] = {'train': train_stats, 'test': test_stats}

# ===============================
# 8️⃣ SECTION-WISE VISUALIZATIONS
# ===============================
for sec in sections:
    df_plot = results_dict[sec]
    plt.figure(figsize=(10,6))
    sns.scatterplot(x='actual_task_minutes', y='predicted_task_minutes', alpha=0.7)
    plt.plot([0, df_plot['actual_task_minutes'].max()],
             [0, df_plot['actual_task_minutes'].max()],
             'r--', linewidth=2)
    plt.title(f"{sec} Actual vs Predicted Task Minutes (2024)")
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.show()

    plt.figure(figsize=(12,5))
    plt.plot(df_plot['date'], df_plot['actual_task_minutes'], label='Actual')
    plt.plot(df_plot['date'], df_plot['predicted_task_minutes'], label='Predicted')
    plt.title(f"{sec} Time Series Plot 2024")
    plt.xlabel("Date")
    plt.ylabel("Task Minutes")
    plt.legend()
    plt.show()

# ===============================
# 9️⃣ RESULTS
# ===============================
# results_dict contains six DataFrames, one per section
# stats_dict contains section-wise stats for train and test


[I 2025-08-23 01:29:34,160] A new study created in memory with name: no-name-cbe319c8-fed0-4cb6-9b4c-414b9373ce04



[Main Loop] Processing section: SEC-001
[Optuna] Tuning section: SEC-001


  0%|          | 0/70 [00:00<?, ?it/s]

01:29:34 - cmdstanpy - INFO - Chain [1] start processing
01:29:34 - cmdstanpy - INFO - Chain [1] done processing
01:29:35 - cmdstanpy - INFO - Chain [1] start processing
01:29:35 - cmdstanpy - INFO - Chain [1] start processing
01:29:35 - cmdstanpy - INFO - Chain [1] start processing
01:29:35 - cmdstanpy - INFO - Chain [1] start processing
01:29:35 - cmdstanpy - INFO - Chain [1] start processing
01:29:35 - cmdstanpy - INFO - Chain [1] start processing
01:29:35 - cmdstanpy - INFO - Chain [1] done processing
01:29:35 - cmdstanpy - INFO - Chain [1] start processing
01:29:35 - cmdstanpy - INFO - Chain [1] start processing
01:29:35 - cmdstanpy - INFO - Chain [1] start processing
01:29:35 - cmdstanpy - INFO - Chain [1] start processing
01:29:36 - cmdstanpy - INFO - Chain [1] done processing
01:29:36 - cmdstanpy - INFO - Chain [1] done processing
01:29:36 - cmdstanpy - INFO - Chain [1] done processing
01:29:36 - cmdstanpy - INFO - Chain [1] done processing
01:29:36 - cmdstanpy - INFO - Chain [

[I 2025-08-23 01:29:36,938] Trial 0 finished with value: 324.8163187189757 and parameters: {'changepoint_prior_scale': 0.0042399146924145675, 'seasonality_prior_scale': 3.262043795414582, 'holidays_prior_scale': 2.9842825350410114}. Best is trial 0 with value: 324.8163187189757.


...

02:17:07 - cmdstanpy - INFO - Chain [1] start processing
02:17:07 - cmdstanpy - INFO - Chain [1] done processing
02:17:08 - cmdstanpy - INFO - Chain [1] start processing
02:17:08 - cmdstanpy - INFO - Chain [1] start processing
02:17:08 - cmdstanpy - INFO - Chain [1] start processing
02:17:09 - cmdstanpy - INFO - Chain [1] start processing
02:17:09 - cmdstanpy - INFO - Chain [1] done processing
02:17:09 - cmdstanpy - INFO - Chain [1] done processing
02:17:09 - cmdstanpy - INFO - Chain [1] done processing
02:17:09 - cmdstanpy - INFO - Chain [1] start processing
02:17:09 - cmdstanpy - INFO - Chain [1] start processing
02:17:09 - cmdstanpy - INFO - Chain [1] start processing
02:17:09 - cmdstanpy - INFO - Chain [1] start processing
02:17:09 - cmdstanpy - INFO - Chain [1] start processing
02:17:09 - cmdstanpy - INFO - Chain [1] start processing
02:17:09 - cmdstanpy - INFO - Chain [1] done processing
02:17:09 - cmdstanpy - INFO - Chain [1] done processing
02:17:09 - cmdstanpy - INFO - Chain [

[I 2025-08-23 02:17:10,211] Trial 69 finished with value: 577.9131150984388 and parameters: {'changepoint_prior_scale': 0.0054051945247293185, 'seasonality_prior_scale': 5.8257876693248365, 'holidays_prior_scale': 28.956425915789453}. Best is trial 32 with value: 530.5179269603154.
SEC-006 best params: {'changepoint_prior_scale': 0.0025178305740946876, 'seasonality_prior_scale': 1.6667875832972159, 'holidays_prior_scale': 7.845337399387722}


02:17:10 - cmdstanpy - INFO - Chain [1] start processing
02:17:10 - cmdstanpy - INFO - Chain [1] done processing


ValueError: Could not interpret value `actual_task_minutes` for `x`. Value is a string, but `data` was not passed.

<Figure size 1000x600 with 0 Axes>